<a href="https://colab.research.google.com/github/lis-r-barreto/ufs-ensef-2025-minicurso-processamento-big-data-spark/blob/main/processamento_big_data_spark_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção de Fraudes em Transações de Cartão de Crédito: Uma Análise com PySpark

A detecção de fraude em transações financeiras é um desafio crítico no setor bancário e de pagamentos. Com o volume massivo de transações diárias, a identificação rápida e precisa de atividades fraudulentas é essencial para minimizar perdas financeiras e manter a confiança dos clientes. Este notebook explora a aplicação do PySpark, uma ferramenta poderosa para processamento de dados em larga escala, na construção de um sistema de detecção de fraude em transações de cartão de crédito. Dada a natureza desbalanceada dos dados de fraude, onde a esmagadora maioria das transações é legítima, abordaremos técnicas específicas para lidar com este desequilíbrio, visando a construção de um modelo eficaz na identificação das raras, porém custosas, ocorrências de fraude.


![](https://imgs.search.brave.com/QB927B8tJyUpWaIXYigepER6BqyOd2hicsD7OxEfsyA/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly9tZWRp/YS5nZXR0eWltYWdl/cy5jb20vaWQvODU1/Mzg4ODg0L3Bob3Rv/L3BpbGUtb2YtY3Jl/ZGl0LWNhcmRzLmpw/Zz9zPTYxMng2MTIm/dz0wJms9MjAmYz1V/WWdxY3hmNjFNNzUx/RGs2VUppUTN2cG1H/OUlmYnZLbm9lUFNZ/OGFLUGxBPQ)

## Sobre o Dataset

Este [dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data) contém transações de cartão de crédito realizadas por portadores de cartões europeus em setembro de 2013. Ele abrange transações que ocorreram em dois dias, totalizando 284.807 transações, das quais apenas 492 são fraudulentas. Isso resulta em um dataset altamente desbalanceado, onde a classe positiva (fraudes) representa apenas 0.172% do total.

As variáveis de entrada são numéricas e, em sua maioria (V1 a V28), são o resultado de uma transformação PCA devido a questões de confidencialidade. As únicas features originais são 'Time' (segundos decorridos desde a primeira transação) e 'Amount' (valor da transação). A feature 'Class' é a variável resposta, indicando fraude (1) ou não fraude (0).

Dada a natureza desbalanceada dos dados, a métrica recomendada para avaliação é a Área Sob a Curva Precision-Recall (AUPRC), pois a acurácia da matriz de confusão pode ser enganosa neste cenário.

Neste notebook, abordaremos este problema de detecção de fraude considerando o desbalanceamento do dataset, utilizando técnicas apropriadas para lidar com ele e avaliando o modelo com métricas relevantes para dados desbalanceados.

In [1]:
!pip --quiet install pyspark kagglehub

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col
import kagglehub

In [3]:
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

## 1. Carregando os Dados

In [4]:
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

csv_file = f"{path}/creditcard.csv"
print("CSV file:", csv_file)

Path to dataset files: /kaggle/input/creditcardfraud
CSV file: /kaggle/input/creditcardfraud/creditcard.csv


In [5]:
try:
  df_csv = spark.read.csv(
      csv_file,
      header=True,
      inferSchema=True
  )
  print("Dataframe criado com sucesso!")
  print("Schema do dataframe:")
  df_csv.printSchema()
  print("Primeiras 5 linhas do dataframe:")
  df_csv.show(5)
except Exception as e:
  print(e)

Dataframe criado com sucesso!
Schema do dataframe:
root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V2

In [6]:
# Define o caminho de saída para o arquivo Parquet em um diretório gravável
parquet_output_path = "/content/creditcard.parquet"

# Escreve o DataFrame no formato Parquet
df_csv.write.parquet(parquet_output_path, mode="overwrite")

print(f"Arquivo CSV convertido para Parquet e salvo em: {parquet_output_path}")

df = spark.read.parquet(parquet_output_path)

Arquivo CSV convertido para Parquet e salvo em: /content/creditcard.parquet


# Detecção de Fraude em Cartão de Crédito com PySpark

Pipeline completo: EDA, balanceamento, seleção de features, treinamento e avaliação usando apenas PySpark.

## 2. EDA Básico

In [7]:
# Tipos de dados e estatísticas
df.printSchema()
df.describe().show()

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

In [8]:
# Checando valores nulos
from pyspark.sql import functions as F
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|Time| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|   0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|     0|    0|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+



In [9]:
# Distribuição das classes
df.groupBy("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



In [10]:
# Estatísticas de Amount por classe
df.groupBy("Class").agg(
    F.count("*").alias("count"),
    F.mean("Amount").alias("mean"),
    F.stddev("Amount").alias("stddev"),
    F.min("Amount").alias("min"),
    F.expr("percentile(Amount, 0.5)").alias("median"),
    F.max("Amount").alias("max")
).show()

+-----+------+------------------+------------------+---+------+--------+
|Class| count|              mean|            stddev|min|median|     max|
+-----+------+------------------+------------------+---+------+--------+
|    1|   492|122.21132113821136|256.68328829771207|0.0|  9.25| 2125.87|
|    0|284315| 88.29102242233286|250.10509222589153|0.0|  22.0|25691.16|
+-----+------+------------------+------------------+---+------+--------+



## 3. Balanceamento do Dataset

In [11]:
# Separando fraudes e não-fraudes
fraud = df.filter(df.Class == 1)
notfraud = df.filter(df.Class == 0)

# Amostrando não-fraudes para balancear
fraud_count = fraud.count()
notfraud_sample = notfraud.sample(False, fraud_count / notfraud.count(), seed=42)

balanced_df = fraud.union(notfraud_sample)
balanced_df.groupBy("Class").count().show()

+-----+-----+
|Class|count|
+-----+-----+
|    1|  492|
|    0|  488|
+-----+-----+



## 4. Seleção de Features por Correlação

In [12]:
# Calculando correlação de cada feature com o target
features = [c for c in balanced_df.columns if c not in ["Class", "Time"]]
correlations = {}
for col in features:
    correlations[col] = balanced_df.stat.corr(col, "Class")

# Selecionando features com |correlação| >= 0.1
selected_features = [col for col, corr in correlations.items() if abs(corr) >= 0.1]
print("Features selecionadas:", selected_features)

Features selecionadas: ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V9', 'V10', 'V11', 'V12', 'V14', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V24', 'V28']


## 5. Treinamento do Modelo

In [13]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=selected_features, outputCol="features")
final_df = assembler.transform(balanced_df).select("features", "Class")

In [14]:
# Split train/test
train, test = final_df.randomSplit([0.8, 0.2], seed=42)

In [15]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="Class", maxIter=10000)
model = lr.fit(train)

## 6. Avaliação do Modelo

In [16]:
predictions = model.transform(test)

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="Class")
auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})

print("Resultados do modelo com Undersampling:")
print(f"Area Under ROC (AUROC): {auroc}")
print(f"Area Under PR (AUPRC): {auprc}")

Resultados do modelo com Undersampling:
Area Under ROC (AUROC): 0.9731250000000005
Area Under PR (AUPRC): 0.9730572542221237
